In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
__author__ = 'Alumno: Edson Castro asignatura Sistemas Distribuidos'

from IDtwitter import * #Modulo con los tokens de twitter.
import twitter
import io
import json
from flask import request
from flask import Flask, render_template
from flask.ext.googlemaps import GoogleMaps
from flask.ext.googlemaps import Map

app = Flask(__name__)
GoogleMaps(app)

def obtenercoordenadas(buscar):
    twitter_api =  oauth_login()
    #Limitamos la busqueda a Espana
    localizacion = "40.2085,-3.713,497mi"

    tweets = twitter_api.search.tweets(q=buscar, count=1000, geocode=localizacion)

    aux = json.dumps(tweets, indent = 1)
    it = json.loads(aux)

    resultado = []
    for i in it['statuses']:
        if i['coordinates'] is not None:
            resultado.append(i['coordinates']['coordinates'][1])
            resultado.append(i['coordinates']['coordinates'][0])

    resultado = zip(resultado[0::1], resultado[1::2])
    #print resultado
    return resultado


#Funcion que muestra el mapa señalando con marcadores donde se han realizado los tweets"
#@app.route("/")
#def mapview():
@app.route("/buscar", methods=['POST'])
def buscar():
    termino = request.form['text'] 
    lista = obtenercoordenadas(termino)
    mymap = Map(
        identifier="view-side",        
        #Coordeandas en el mapa sobre España.
        lat=40.2106414,
        lng=-4.1370476,        
        zoom=6,
        markers=lista,
        style="height:800px;width:800px;margin:0;"
    ) 
    return render_template('mapa.html', mymap=mymap)

@app.route("/")
def index():
    return render_template('index.html')

if __name__ == "__main__":
    app.run(debug=False)